# 1. SARIMA model - PROD

### We are here trying the Holt-Winter ETS model for forecast

In [1]:
# Library Import

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import timeit

from statsmodels.tsa.api import SARIMAX
from math import sqrt
from sklearn.metrics import mean_squared_error

In [2]:
# Creating list of values to 
time = pd.Timestamp("00:00:00")
time_list = []
time_list.append(str(time)[-8:])

# Adding 15 minutes every time
#for i in range(1):
for i in range(24*4-1):
    time += pd.Timedelta(minutes=15)
    time_list.append(str(time)[-8:])
print(time_list)

['00:00:00', '00:15:00', '00:30:00', '00:45:00', '01:00:00', '01:15:00', '01:30:00', '01:45:00', '02:00:00', '02:15:00', '02:30:00', '02:45:00', '03:00:00', '03:15:00', '03:30:00', '03:45:00', '04:00:00', '04:15:00', '04:30:00', '04:45:00', '05:00:00', '05:15:00', '05:30:00', '05:45:00', '06:00:00', '06:15:00', '06:30:00', '06:45:00', '07:00:00', '07:15:00', '07:30:00', '07:45:00', '08:00:00', '08:15:00', '08:30:00', '08:45:00', '09:00:00', '09:15:00', '09:30:00', '09:45:00', '10:00:00', '10:15:00', '10:30:00', '10:45:00', '11:00:00', '11:15:00', '11:30:00', '11:45:00', '12:00:00', '12:15:00', '12:30:00', '12:45:00', '13:00:00', '13:15:00', '13:30:00', '13:45:00', '14:00:00', '14:15:00', '14:30:00', '14:45:00', '15:00:00', '15:15:00', '15:30:00', '15:45:00', '16:00:00', '16:15:00', '16:30:00', '16:45:00', '17:00:00', '17:15:00', '17:30:00', '17:45:00', '18:00:00', '18:15:00', '18:30:00', '18:45:00', '19:00:00', '19:15:00', '19:30:00', '19:45:00', '20:00:00', '20:15:00', '20:30:00', '20

In [23]:
seasonal=[60]

models = list()
# define config lists
p_params = [0, 1, 2]
d_params = [0, 1]
q_params = [0, 1, 2]
t_params = [ 'n' , 'c' , 't' , 'ct' ]
P_params = [0, 1, 2]
D_params = [0, 1]
Q_params = [0, 1, 2]
m_params = seasonal
# create config instances
for p in p_params:
    for d in d_params:
        for q in q_params:
            for t in t_params:
                for P in P_params:
                    for D in D_params:
                        for Q in Q_params:
                            for m in m_params:
                                cfg = [(p,d,q), (P,D,Q,m), t]
                                models.append(cfg)

In [24]:
models

[[(0, 0, 0), (0, 0, 0, 60), 'n'],
 [(0, 0, 0), (0, 0, 1, 60), 'n'],
 [(0, 0, 0), (0, 0, 2, 60), 'n'],
 [(0, 0, 0), (0, 1, 0, 60), 'n'],
 [(0, 0, 0), (0, 1, 1, 60), 'n'],
 [(0, 0, 0), (0, 1, 2, 60), 'n'],
 [(0, 0, 0), (1, 0, 0, 60), 'n'],
 [(0, 0, 0), (1, 0, 1, 60), 'n'],
 [(0, 0, 0), (1, 0, 2, 60), 'n'],
 [(0, 0, 0), (1, 1, 0, 60), 'n'],
 [(0, 0, 0), (1, 1, 1, 60), 'n'],
 [(0, 0, 0), (1, 1, 2, 60), 'n'],
 [(0, 0, 0), (2, 0, 0, 60), 'n'],
 [(0, 0, 0), (2, 0, 1, 60), 'n'],
 [(0, 0, 0), (2, 0, 2, 60), 'n'],
 [(0, 0, 0), (2, 1, 0, 60), 'n'],
 [(0, 0, 0), (2, 1, 1, 60), 'n'],
 [(0, 0, 0), (2, 1, 2, 60), 'n'],
 [(0, 0, 0), (0, 0, 0, 60), 'c'],
 [(0, 0, 0), (0, 0, 1, 60), 'c'],
 [(0, 0, 0), (0, 0, 2, 60), 'c'],
 [(0, 0, 0), (0, 1, 0, 60), 'c'],
 [(0, 0, 0), (0, 1, 1, 60), 'c'],
 [(0, 0, 0), (0, 1, 2, 60), 'c'],
 [(0, 0, 0), (1, 0, 0, 60), 'c'],
 [(0, 0, 0), (1, 0, 1, 60), 'c'],
 [(0, 0, 0), (1, 0, 2, 60), 'c'],
 [(0, 0, 0), (1, 1, 0, 60), 'c'],
 [(0, 0, 0), (1, 1, 1, 60), 'c'],
 [(0, 0, 0), (

In [17]:
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

# create a set of exponential smoothing configs to try
def exp_smoothing_configs(seasonal=[None]):
    models = list()
    # define config lists
    p_params = [0, 1, 2]
    d_params = [0, 1]
    q_params = [0, 1, 2]
    t_params = [ 'n' , 'c' , 't' , 'ct' ]
    P_params = [0, 1, 2]
    D_params = [0, 1]
    Q_params = [0, 1, 2]
    m_params = seasonal
    # create config instances
    for p in p_params:
        for d in d_params:
            for q in q_params:
                for t in t_params:
                    for P in P_params:
                        for D in D_params:
                            for Q in Q_params:
                                for m in m_params:
                                    cfg = [(p,d,q), (P,D,Q,m), t]
                                    models.append(cfg)

# one-step Holt Winters Exponential Smoothing forecast
def sarima_forecast_grid_search(train, test, configurations):
    best_config = ""
    best_rmse = 99999
    for i in range(len(configurations)):
        try:
            order, sorder, trend = configurations[i]
            
            # define model
            model = SARIMAX(train, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False).fit(disp=False)
            # predict n values, the length of test arry
            pred = model.predict(start=len(train)+1,end=len(test)+len(train))
            temp_indicator = measure_rmse(test,pred)
            if temp_indicator < best_rmse:
                best_rmse = temp_indicator
                best_config = configurations[i]
        except:
            pass
    return(best_config, best_rmse)

In [4]:
def model_generation(time, time_step, config):

    # I. INITIALIZATION OF THE MODEL
    
    start_time = timeit.default_timer()
    # Timestamp of when we want to start the prediction
    target_start = time
    # n-1 minutes before the target date and n min after
    time_variation = time_step
    # Loading from github the dataset. It is an extract of the madeleine station only
    df_station = pd.read_csv("https://raw.githubusercontent.com/alexisgcomte/velib-prediction/master/3.%20Modeling%20Research/datasets/madeleine_weekday.csv")

    # II. PREPROCESSING PIPE
    # Removing first not useless column
    df_station.pop("Unnamed: 0")

    # Adding day and minute
    df_station["day"] = df_station.date.apply(lambda x: x[:10])

    # Converting to datetime the number of availiable docks
    df_station['date'] = pd.to_datetime(df_station['date'])

    # Variable declaration

    # list the different days. Removing 2020_01_15 for lack of data.
    day_list = df_station.day.unique()

    #removing first day (incomplete data) and last day (only midnight)
    day_list = day_list[1:-1]

    # Creation of time bounds
    lower_time_bound = str(pd.Timestamp(target_start) - pd.Timedelta(minutes=time_variation))[-8:]
    upper_time_bound = str(pd.Timestamp(target_start) + pd.Timedelta(minutes=time_variation+1))[-8:]

    # proper consolidation
    df_conso = pd.DataFrame(columns=df_station.columns)
    for i in range(len(day_list)-1):

        # The 30 min after the time, the day before

        day_lower = day_list[i]
        df_day_lower = df_station[df_station["day"] == day_lower]
        df_day_lower = df_day_lower[(df_day_lower["date"] > "{} {}".format(day_lower, target_start))
                    & (df_day_lower["date"] < "{} {}".format(day_lower,upper_time_bound))]


        # The 30 min before the time, the day 


        day_upper = day_list[i+1]
        df_day_upper = df_station[df_station["day"] == day_upper]
        df_day_upper = df_day_upper[(df_day_upper["date"] > "{} {}".format(day_upper, lower_time_bound))
                    & (df_day_upper["date"] <= "{} {}".format(day_upper,target_start))]

        df_conso = pd.concat([df_conso, df_day_lower, df_day_upper])

    # Backtest for ulterior use
    day_backtest = day_list[len(day_list)-1]
    df_day_lower = df_station[df_station["day"] == day_backtest]
    df_day_lower = df_day_lower[(df_day_lower["date"] > "{} {}".format(day_backtest, target_start))
                & (df_day_lower["date"] < "{} {}".format(day_backtest,upper_time_bound))]["availiable_docks"]
    df_backtest = np.array(df_day_lower)



    df_conso = df_conso.reset_index(drop="True")

    # Converting to int the number of availiable docks
    df_conso["availiable_docks"] = df_conso["availiable_docks"].astype(int)
    df_conso.head()

    # Creating time_serie
    time_serie = np.array(df_conso["availiable_docks"])

    # Doing a slit of aust between train and test
    split_ratio = 1- 1/(len(time_serie)/(time_variation*2))
    train = time_serie[0:int(round(len(time_serie)*split_ratio,0))]
    test = time_serie[int(round(len(time_serie)*split_ratio,0)):]


    # This will be our performance indicator
    

    cfg_list = config
    
    # B. Finding the best parameters over one periode
    best_parameters, best_rmse = sarima_forecast_grid_search(train, test, cfg_list)
    # Loading the best parameters found befor
    t,d,s,p,b,r = best_parameters

    # Creating the model on full time serie
    model = SARIMAX(train, trend=t, damped=d, seasonal=s, seasonal_periods=p).fit(optimized=True,use_boxcox=b,remove_bias=r)


    # C. Training on full time serie to optimize before prediction¶

    # Creating the model on full time serie, using best parameters
    model = SARIMAX(time_serie, trend=t, damped=d, seasonal=s, seasonal_periods=p).fit(optimized=True,use_boxcox=b,remove_bias=r)

    # predicting on all time serie for global RMSE
    pred_model = model.predict(1,end=len(time_serie))
    RMSE_test = measure_rmse(time_serie,pred_model)


    # Prediction the dataset + time variation
    final_prediction = model.predict(start=(len(time_serie)+1), end=(len(time_serie)+time_variation))


    # IV - Making the prediction up to the time variation and creating assessment dataframe

    # Creating a dataframe for assessement
    df_assessement = pd.DataFrame(columns=["actual","forecast_first_run","forecast_optimized","forecast_optimized_round","backtest"])
    # Setting actual at the last known value
    df_assessement.actual = (np.concatenate((time_serie, (np.zeros(time_variation)+time_serie[-1:])), axis = 0))
    # Setting forecast at time_serie for the past and the final prediction for the futur
    df_assessement.forecast_optimized = (np.concatenate((time_serie, final_prediction), axis = 0))
    df_assessement.forecast_optimized_round = df_assessement.forecast_optimized.apply(lambda x:round(x,0))
    df_assessement.backtest = (np.concatenate((time_serie, df_backtest), axis = 0))
    df_assessement["date"] = df_conso["date"]
    # Making a synthetic dataset for easier visualisation

    df_assessement_small = df_assessement[-time_variation*2:]

    # V. SYNTHSIS

    # Time to run

    elapsed = timeit.default_timer() - start_time


    # RMSE for assessement

    rmse_f_b = measure_rmse(df_backtest,df_assessement_small.forecast_optimized[time_variation:])
    rmse_rf_b = measure_rmse(df_backtest,df_assessement_small.forecast_optimized_round[time_variation:])
    #reminder of best parameters

    results = [str(target_start)[-8:], rmse_f_b, rmse_rf_b, RMSE_test, best_parameters, elapsed]
    return results

In [21]:
config = exp_smoothing_configs(seasonal=[60])

In [ ]:
result = []
step =0
for i in time_list:
    step += 1
    try:
        temp = model_generation(i, 30, models[2:5])
        result.append(temp)
        print(temp)
    except:
        pass
    print("progression:",step,"/",len(time_list))
    

In [11]:
result

[]